In [1]:
import numpy as np
import pandas as pd
from faker import Faker
from scipy.stats import skewnorm

# 1. Corregir distribución de severidad según GEMA 5.0
GEMA_SEVERITY_DIST = {
    "Intermittent": 0.25,
    "Mild Persistent": 0.35,
    "Moderate Persistent": 0.25,
    "Severe Persistent": 0.15
}

# 2. Parámetros de adherencia realistas (estudio REALISE)
SYMBICORT_ADHERENCE_PARAMS = {
    "Intermittent": (1.8, 2.2),
    "Mild Persistent": (2.5, 2.5),
    "Moderate Persistent": (3.2, 1.8),
    "Severe Persistent": (3.5, 1.2)
}

PERIOD = 24 * 365

def generate_gema_cohort(num_patients=1000):
    fake = Faker('es_ES')
    np.random.seed(42)
    
    # 3. Generar edades realistas con clip médico
    ages = np.clip(
        skewnorm.rvs(4, loc=35, scale=15, size=num_patients).astype(int),
        12,  # Edad mínima para diagnóstico claro
        90   # Edad máxima realista
    )
    
    patients = pd.DataFrame({
        "patient_id": [f"AZ-{i:04d}-CAT" for i in range(num_patients)],
        "gender": np.random.choice(["M", "F"], num_patients, p=[0.45, 0.55]),
        "age": ages,
        "postal_code": [fake.postcode()[:5] for _ in range(num_patients)],
        "gema_severity": np.random.choice(
            list(GEMA_SEVERITY_DIST.keys()),
            num_patients,
            p=list(GEMA_SEVERITY_DIST.values())
        )
    })
    
    # 4. Corregir función de adherencia con groupby correcto
    def _get_adherence(severity_group):
        severity = severity_group.name  # Acceder al nombre del grupo
        a, b = SYMBICORT_ADHERENCE_PARAMS[severity]
        return np.clip(np.random.beta(a, b, size=len(severity_group)), 0.3, 0.95)
    
    patients["symbicort_adherence"] = patients.groupby("gema_severity")["gema_severity"].transform(_get_adherence)
    
    # 5. Riesgo de exacerbación basado en EMA
    exacerbation_risk = patients["gema_severity"].map({
        "Intermittent": 0.12,
        "Mild Persistent": 0.27,
        "Moderate Persistent": 0.43,
        "Severe Persistent": 0.68
    })
    patients["base_exacerbation_risk"] = exacerbation_risk * (1.15 - patients["symbicort_adherence"])
    
    # 6. Comorbilidades con lógica clínica
    patients["has_allergic_rhinitis"] = np.where(
        patients["gema_severity"].isin(["Moderate Persistent", "Severe Persistent"]),
        np.random.binomial(1, 0.75, num_patients),
        np.random.binomial(1, 0.45, num_patients)
    )
    
    # 7. COPD con modelo de edad realista
    patients["has_COPD"] = np.where(
        patients["age"] > 40,
        np.random.binomial(1, np.clip(0.015 * (patients["age"] - 40), 0, 0.35)),
        0
    )
    
    # 8. Generación de eventos de inhalador optimizada
    timestamps = pd.date_range(
        start="2024-01-01",
        periods=PERIOD,
        freq='h',
        tz='Europe/Madrid'
    )
    
    base_puffs_map = {
        "Intermittent": 0.25,
        "Mild Persistent": 0.55,
        "Moderate Persistent": 1.35,
        "Severe Persistent": 2.75
    }
    
    # 9. Vectorización para mejor performance
    circadian = np.sin(2 * np.pi * np.arange(PERIOD) / 24 - np.pi/2) * 0.35 + 1
    circadian = np.clip(circadian, 0.4, 1.6)  # Límites fisiológicos
    
    inhaler_dfs = []
    for severity, group in patients.groupby("gema_severity"):
        n_patients = len(group)
        base_puffs = base_puffs_map[severity]
        
        # 10. Generación vectorizada de puffs
        puffs = np.random.poisson(
            lam=(base_puffs * circadian.reshape(1, -1) * 
                 group["symbicort_adherence"].values.reshape(-1, 1)),
            size=(n_patients, PERIOD)
        )
        
        # 11. Generación geoespacial consistente
        base_lat = 41.3851 + np.random.normal(0, 0.003, n_patients)
        base_lon = 2.1734 + np.random.normal(0, 0.003, n_patients)
        
        severity_df = pd.DataFrame({
            "patient_id": np.repeat(group["patient_id"], PERIOD),
            "timestamp": np.tile(timestamps, n_patients),
            "puffs": puffs.flatten(),
            "latitude": np.repeat(base_lat, PERIOD),
            "longitude": np.repeat(base_lon, PERIOD),
            "device_type": "Symbicort Turbuhaler"
        })
        
        inhaler_dfs.append(severity_df)
    
    return patients, pd.concat(inhaler_dfs, ignore_index=True)

# 12. Guardado optimizado para AWS HealthLake
patients, inhaler_events = generate_gema_cohort(1000)

# Particionado por fecha para mejor performance en Athena
inhaler_events["date"] = inhaler_events["timestamp"].dt.date
inhaler_events.to_parquet(
    "../data/raw/iot_inhaler/inhaler_events.parquet",
    partition_cols=["date"],
    compression='snappy'
)
patients.to_parquet(
    "../data/raw/iot_inhaler/patients.parquet",
    compression='snappy'
)

In [2]:
def clinical_validation(patients_df):
    # 1. Definir orden categórico estricto
    gema_order = [
        "Intermittent", 
        "Mild Persistent", 
        "Moderate Persistent", 
        "Severe Persistent"
    ]
    
    patients_df["gema_severity"] = pd.Categorical(
        patients_df["gema_severity"],
        categories=gema_order,
        ordered=True
    )
    
    # 2. Obtener distribución manteniendo orden original
    severity_counts = patients_df["gema_severity"].value_counts(normalize=True, sort=False)
    
    # 3. Comparación ordenada con tolerancia ajustada
    expected_dist = np.array(list(GEMA_SEVERITY_DIST.values()))
    obtained_dist = severity_counts.values
    
    assert np.allclose(
        obtained_dist,
        expected_dist,
        atol=0.035,  # Tolerancia para N=1000
        rtol=0.05    # Tolerancia relativa adicional
    ), f"""Distribución fuera de rango esperado:
    Esperado: {dict(zip(gema_order, expected_dist))}
    Obtenido: {dict(zip(gema_order, obtained_dist))}"""
    
    # Resto de validaciones...
    print("✓ Validación exitosa")

clinical_validation(patients)

✓ Validación exitosa
